In [1]:
from torch.utils.data import DataLoader
from model_utilities.models.cifar_resnet import resnet18_3x3, \
    ResNet18_3x3_Weights
from torchvision.datasets import CIFAR10
from pathlib import Path


home = Path.home()
dataset = CIFAR10(home / "data", train=True, download=True,
                  transform=ResNet18_3x3_Weights.DEFAULT.transforms())
loader = DataLoader(dataset, batch_size=64, shuffle=True)

base_model = resnet18_3x3()

In [2]:
from torchwatcher.interjection import WrappedForwardBackwardInterjection, \
    interject_by_name


class GradTracker(WrappedForwardBackwardInterjection):
    def process(self, name, module, input, output):
        pass

    def process_backward(self, name, module, grad_input, grad_output):
        # print("backward",
        #       name,
        #       grad_input.shape if grad_input is not None else None)
        # print(dict(module.named_parameters()))
        print([g.grad.norm() for g in module.parameters()])


model = interject_by_name(base_model, "layer3.1.conv1", GradTracker())

In [3]:
import torch
from torchbearer import Trial

optim = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss()
trial = Trial(model, optim, criterion, verbose=2, metrics=['loss', 'accuracy'])

trial.train()
trial.to("mps")
trial.with_train_generator(loader).run(1)

# for n, p in model.named_parameters():
    # if "layer3.1.conv1" in n:
    # print(n, p.device)

0/1(t):   0%|          | 0/782 [00:00<?, ?it/s]

[tensor(1.5640, device='mps:0')]
[tensor(1.5104, device='mps:0')]
[tensor(1.4570, device='mps:0')]
[tensor(1.2270, device='mps:0')]
[tensor(0.9981, device='mps:0')]
[tensor(0.9191, device='mps:0')]
[tensor(0.7861, device='mps:0')]
[tensor(0.6828, device='mps:0')]
[tensor(0.6477, device='mps:0')]
[tensor(0.6531, device='mps:0')]
[tensor(0.5462, device='mps:0')]
[tensor(0.5828, device='mps:0')]
[tensor(0.5032, device='mps:0')]
[tensor(0.5494, device='mps:0')]
[tensor(0.4572, device='mps:0')]
[tensor(0.4393, device='mps:0')]
[tensor(0.4650, device='mps:0')]
[tensor(0.4277, device='mps:0')]
[tensor(0.3759, device='mps:0')]
[tensor(0.4118, device='mps:0')]
[tensor(0.4127, device='mps:0')]
[tensor(0.3724, device='mps:0')]
[tensor(0.3961, device='mps:0')]
[tensor(0.3682, device='mps:0')]
[tensor(0.3865, device='mps:0')]
[tensor(0.3539, device='mps:0')]
[tensor(0.3618, device='mps:0')]
[tensor(0.3577, device='mps:0')]
[tensor(0.3753, device='mps:0')]
[tensor(0.3984, device='mps:0')]
[tensor(0.

[{'running_loss': 0.8884350657463074,
  'running_acc': 0.6899999976158142,
  'loss': 1.1999280452728271,
  'acc': 0.5786399841308594,
  'train_steps': 782,
  'validation_steps': None}]